<a href="https://colab.research.google.com/github/Lambda-School-Labs/Labs26-Citrics-DS-TeamC/blob/mtoce/notebooks/Copy_of_combined_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import json

In [ ]:
# MOUNT Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# PATH and load info
base_path = '/content/drive/My Drive/lambda_school/driftly_data/'

weather = pd.read_csv(base_path + 'weather_data/weather_insert_final.csv')
housing = pd.read_csv(base_path + 'housing_data/housing_merge.csv')
jobs = pd.read_csv(base_path + 'jobs_data/jobs_merge.csv')
covid = pd.read_csv(base_path + 'covid_data/covid_merge.csv')
pop = pd.read_csv(base_path + 'population_data/city_pop.csv')
keys = pd.read_csv(base_path + 'database_tables/keys_pop.csv')

# Geography

In [ ]:
geographic_breakdown = pd.read_csv(base_path + 'population_data/geographic_breakdown.csv')
geographic_breakdown.drop(columns=['Unnamed: 0'], inplace=True)

## CHECK:
print(geographic_breakdown.shape)
geographic_breakdown.head()

In [ ]:
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy.types import Integer, Float, Text, String, DateTime

In [ ]:
db_flavor = "postgres"
db_python_library = "psycopg2"
db_username = "postgres"
db_password = "UM*3tAe<ns!q!nG3"
db_host = "citrics-ds-teamc.cav8gkdxva9e.us-east-1.rds.amazonaws.com"
db_port = "5432"
db_name = "postgres"
db_uri = db_flavor + "+" + db_python_library + "://" + db_username + ":" + db_password + "@" + db_host + ":" + db_port + "/" + db_name

# CREATE engine
engine = create_engine(db_uri, echo=True)

In [ ]:
geographic_breakdown.dtypes

In [ ]:
# table_name = 'geographic_breakdown'

# geographic_breakdown.to_sql(table_name, engine, if_exists='replace', index=False, chunksize=500,
#             dtype={
#                 "region_id": Integer,
#                 "postal": String,
#                 "metro": String,
#                 "county": String,
#                 "city": String,
#                 "region_type": String,
#                 "city-state": String,
#                 "county-state": String,
#                 "region_info": String
#             })

In [ ]:
geo_dictionary = {}

for postal in geographic_breakdown['postal']:
    geo_dictionary[postal] = {}

for idx, val in enumerate(geographic_breakdown['county_state']):
    postal = val[-2:]
    if postal in geo_dictionary:
        geo_dictionary[postal][val] = set()

for idx, val in enumerate(geographic_breakdown['city_state']):
    postal = val[-2:]
    county_state = geographic_breakdown['county_state'][idx]
    
    if postal in geo_dictionary:
        if county_state in geo_dictionary[postal]:
            geo_dictionary[postal][county_state].add(val) 


# comprehensive_dictionary
geo_dictionary['NY']

In [ ]:
geo_regionid_lst = geographic_breakdown['region_id'].to_list()
# len(geo_regionid_lst)

# Real Estate

In [ ]:
ZATT = pd.read_csv(base_path + 'housing_data/ZATT_data_full.csv')
ZATT.drop(columns=['Unnamed: 0'], inplace=True)

## CHECK:
print(ZATT.shape)
ZATT.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(6010825, 14)


,indicator_id,region_id,region_type,date,year,month,value,city,county,metro,postal,region_info,city_state,county_state
0,ZATT,6181,city,2020-07-31,2020,2020-07,1143222.0,New York,Queens County,New York-Newark-Jersey City,NY,New York; NY; New York-Newark-Jersey City; Que...,"New York, NY","Queens County, NY"
1,ZATT,6181,city,2020-06-30,2020,2020-06,1143665.0,New York,Queens County,New York-Newark-Jersey City,NY,New York; NY; New York-Newark-Jersey City; Que...,"New York, NY","Queens County, NY"
2,ZATT,6181,city,2020-05-31,2020,2020-05,1143204.0,New York,Queens County,New York-Newark-Jersey City,NY,New York; NY; New York-Newark-Jersey City; Que...,"New York, NY","Queens County, NY"
3,ZATT,12447,city,2020-07-31,2020,2020-07,1492845.0,Los Angeles,Los Angeles County,Los Angeles-Long Beach-Anaheim,CA,Los Angeles; CA; Los Angeles-Long Beach-Anahei...,"Los Angeles, CA","Los Angeles County, CA"
4,ZATT,12447,city,2020-06-30,2020,2020-06,1476789.0,Los Angeles,Los Angeles County,Los Angeles-Long Beach-Anaheim,CA,Los Angeles; CA; Los Angeles-Long Beach-Anahei...,"Los Angeles, CA","Los Angeles County, CA"


In [ ]:
def RE_json_func(ZATT, year_month_day, geo_regionid_lst):
    
    ## subsetting df for columns
    ZATT_subset = ZATT.copy()
    ZATT_subset = ZATT_subset[ZATT_subset['region_id'].isin(geo_regionid_lst)]
    ZATT_subset = ZATT_subset[['date', 'indicator_id', 'region_id', 'city', 'postal', 'city_state', 'value']]

    ## subsetting df for date
    df_date = ZATT_subset[ZATT_subset['date'] >= year_month_day]
    df_date.reset_index(inplace=True)

    ## creating storage dictionary
    state_cities_dictionary = {}
            
    idx = 0
    for i in df_date['postal']:
        if i not in state_cities_dictionary:
            state_cities_dictionary[i] = []
        else:
            zip_ = [df_date['city_state'][idx], df_date['region_id'][idx], df_date['value'][idx]]
            state_cities_dictionary[i].append(zip_)
        idx += 1


    ## inserting values into storage dictionary
    for i in state_cities_dictionary:
        city_state_combo = [lst[0] for lst in state_cities_dictionary[i]]   
        value = [lst[2] for lst in state_cities_dictionary[i]]
        
        combo_value_dict = dict(zip(city_state_combo, value))

        state_cities_dictionary[i] = combo_value_dict

    ## converting to JSON
    final_states_city_dictionary = json.dumps(state_cities_dictionary)
    
    ## return: final_states_city_dictionary
    return final_states_city_dictionary


## CHECK:
RE_JSON = RE_json_func(ZATT, '2020-07-31', geo_regionid_lst)
# RE_JSON

# Jobs

In [ ]:
# Wage / Earnings Data
wage_data = pd.read_csv(r'.\combo_data\jobs\quarterly_data_copy.csv')

# CHECK:
print(wage_data.shape)
wage_data.head()

FileNotFoundError: ignored

In [ ]:
def wage_func(df, year, quarter):
    
    ## Copying and cleaning df
    X = df.copy()
    
    X = X[(X['Year'] == year) &
          (X['Qtr'] == quarter)]

    X.reset_index(inplace=True)
    X.drop(columns=['index', 'Unnamed: 0'], inplace=True)


    ## POSTAL DICT:
    postal_dict = {}

    idx = 0
    for i in X['postal']:
        postal_dict[i] = {}

    for i in X['county_state']:
        if i[-2:] in postal_dict:
            postal_dict[i[-2:]][i] = {}


    ## COUNTY DICT:
    county_state_dict = {}
    for i in X['county_state']:
        county_state_dict[i] = {}

    for idx, ind in enumerate(X['Industry']):
        county_state = X['county_state'][idx]
        if county_state in county_state_dict:
            county_state_dict[county_state][ind] = {}


    # GROUPBY
    df_groupby = X.groupby(['county_state','Industry']).mean()

    # creating groupby dictionary
    groupby_dict = df_groupby.to_dict('index')

    # looping through the dictionary and substituting dictionary values
    for county_state in county_state_dict:
        for industry in county_state_dict[county_state]:
                county_state_dict[county_state][industry] = groupby_dict[county_state, industry]

    # looping through the postal_dict and replacing the value item with the supplemental county_state dict value item
    for i in postal_dict:
        countstate_dict = postal_dict[i]
        for j in countstate_dict:
            if j in county_state_dict:
                countstate_dict[j] = county_state_dict[j]

    # reutrn
    return postal_dict


# CHCEK:
wage_dict = wage_func(wage_data, 2019, 3)
# wage_dict['NY']['New York, NY']

In [ ]:
# Business Dynmaics Data
biz_dyn_national = pd.read_csv(r'.\combo_data\jobs\jobs_business_dynamics_copy.csv')
biz_dyn_national.drop(columns='Unnamed: 0', inplace=True)

# CHECK:
print(biz_dyn_national.shape)
biz_dyn_national.head()

In [ ]:
# National JOLTS Data
jolts_national = pd.read_csv(r'.\combo_data\jobs\jobs_jolts_national_copy.csv')
jolts_national.drop(columns='Unnamed: 0', inplace=True)

# CHECK:
print(jolts_national.shape)
jolts_national.head()

In [ ]:
# Industry JOLTS Data
jolts_industry = pd.read_csv(r'.\combo_data\jobs\jobs_jolts_industry_copy.csv')
jolts_industry.drop(columns='Unnamed: 0', inplace=True)

# CHECK:
print(jolts_industry.shape)
jolts_industry.head()

In [ ]:
# Local Area and Unemployment Data
lau_state_df = pd.read_csv(r'.\combo_data\jobs\jobs_lau_state_copy.csv')
lau_state_df.drop(columns='Unnamed: 0', inplace=True)

# CHECK:
print(lau_state_df.shape)
lau_state_df.head()

# Weather

In [ ]:
# weather_data = pd.read_csv(r'.\combo_data\weather_data_final.csv')
weather_data = weather.copy()
## CHECK:
# print(weather_data.shape)
# weather_data.head()
weather_data.head()

In [ ]:
# Creating a single function for ALL the weather data
def weather_json_func(df):
    
    ## PRELIMINARY
    def convert_location_to_state_plus_postal(data):
        # us state abbr list for capitals with "-" in between cities
        us_state_abbrev = {'Alabama': 'AL','Alaska': 'AK','American Samoa': 'AS','Arizona': 'AZ','Arkansas': 'AR','California': 'CA','Colorado': 'CO','Connecticut': 'CT','Delaware': 'DE','District-of-Columbia': 'DC','Florida': 'FL','Georgia': 'GA','Guam': 'GU','Hawaii': 'HI','Idaho': 'ID','Illinois': 'IL','Indiana': 'IN','Iowa': 'IA','Kansas': 'KS','Kentucky': 'KY','Louisiana': 'LA','Maine': 'ME','Maryland': 'MD','Massachusetts': 'MA','Michigan': 'MI','Minnesota': 'MN','Mississippi': 'MS','Missouri': 'MO','Montana': 'MT','Nebraska': 'NE','Nevada': 'NV','New-Hampshire': 'NH','New-Jersey': 'NJ','New-Mexico': 'NM','New-York': 'NY','North-Carolina': 'NC','North-Dakota': 'ND','Northern-Mariana-Islands':'MP','Ohio': 'OH','Oklahoma': 'OK','Oregon': 'OR','Pennsylvania': 'PA','Puerto-Rico': 'PR','Rhode-Island': 'RI','South-Carolina': 'SC','South-Dakota': 'SD','Tennessee': 'TN','Texas': 'TX','Utah': 'UT','Vermont': 'VT','Virgin-Islands': 'VI','Virginia': 'VA','Washington': 'WA','West-Virginia': 'WV','Wisconsin': 'WI','Wyoming': 'WY'}
    
        # split location between city and state and save to "new" df
        new = data["location"].str.split("_", n = 1, expand = True)

        # create "city" and "state" cols in data df based on "new" df
        data["city"] = new[0] 
        data["state"] = new[1]

        # map postals over state column and save to new postals column
        data["postal"] = data["state"].map(us_state_abbrev)

        # concat city + _ +  postal 
        data['location'] = data['city'] + "_" + data['postal']

        # drop unnecessary cols
        data = data.drop(['city', 'state', 'postal'], axis = 1)

        return data

    # calling function
    # weather_data = convert_location_to_state_plus_postal(df)
    
    
    ## PART 1:
    # clean function
    def clean(DataFrame):
        # copy frame
        X = DataFrame.copy()

        # drop unneeded cols
        X = X.drop(['Unnamed: 0','winddirDegree','moonrise','moonset','moon_illumination','sunrise','sunset','sunHour','tempC'], axis=1)

        # clean out this data, since there are issues with it
        X = X[X['mintempC'] != 0]
        X = X[X['maxtempC'] != 0]

        # create temp df to str split location into multiple cols then concat new DF
        loc_temp = X['location'].str.split('_', expand=True)
        loc_temp = loc_temp.rename(columns={0: "city", 1: "state_abbr"})
        X = pd.concat([X, loc_temp], axis=1)
        X = X.drop('location', axis=1)

        # rename cols from str.split
        X = X.rename(columns={0: "city", 1: "state_abbr"})

        # convert new columns to strings
        X['city'] = X['city'].astype('string')
        X['state_abbr'] = X['state_abbr'].astype('string')

        # replace dashes with spaces in city col
        X['city'] = X['city'].str.replace('-', ' ')

        # split date_time into categories for splitting df seasonally
        X['date_time'] = pd.to_datetime(X['date_time'], infer_datetime_format=True)
        X['year'] = X['date_time'].dt.year
        X['month'] = X['date_time'].dt.month
        X['day'] = X['date_time'].dt.day

        # create cols for fahrenheit from temp_c cols
        X['maxtempF'] = X['maxtempC'] * (9/5) + 32
        X['mintempF'] = X['mintempC'] * (9/5) + 32
        X['FeelsLikeF'] = X['FeelsLikeC'] * (9/5) + 32

        # adding new column 
        X['city_state'] = X['city'] + ", " + X['state_abbr']

        # ordering the columns in the desired way
        cols = ['city', 'state_abbr', 'city_state', 'maxtempF', 'FeelsLikeF', 
                'mintempF', 'maxtempC', 'FeelsLikeC', 'mintempC', 'precipMM', 
                'totalSnow_cm', 'uvIndex', 'DewPointC', 'HeatIndexC', 'WindChillC', 
                'WindGustKmph', 'cloudcover', 'humidity', 'pressure', 'visibility', 
                'windspeedKmph', 'year', 'month', 'day', 'date_time']

        # setting column order
        X = X[cols]

        new_col_names = ['City', 'Postal', 'City_State', 'MaxTempF', 'FeelsLikeF', 
                         'MinTempF', 'MaxTempC', 'FeelsLikeC', 'MinTempC', 'Precip_mm', 
                         'TotalSnow_cm', 'UVindex', 'DewPointC', 'HeatIndexC', 'WindChillC', 
                         'WindGust_kmph', 'CloudCover', 'Humidity', 'Pressure', 'Visibility', 
                         'WindSpeed_kmph', 'year', 'month', 'day', 'date_time']

        cols_new_cols_dict = dict(zip(cols, new_col_names))

        X.rename(columns = cols_new_cols_dict, inplace=True)

        return X
    ## CALLING FUNCTION: clean
    # calling the clean function on the df 
    # data = clean(weather_data)
    data = weather_data
    ## PART 2:
    # Creating the final dictionary that will get converted to JSON that will hold
    # all the relevant weather data
    Final_Weather_Data_dict = {}
      
    for i in data['Postal']:
        if i not in Final_Weather_Data_dict:
            Final_Weather_Data_dict[i] = {}


    ## PART 3:
    # Creating a dictionary that holds the City_State as the key and the Postal
    # code as the value 
    def Postal_City_State_Weather_dict_func(df, Final_Weather_dict):
        # Grouping the data by City_State, and getting the column means
        City_State_groupby = df.groupby('City_State').mean()
        # Creating a list of the City_State_groupby_indices
        City_State_groupby_indices = City_State_groupby.index.to_list()

        # Instantiating a new dictionary to hold the City_States as a key, and the 
        # Postal codes as values
        City_State_Postal_dict = {}
        # Looping through the City_States_groupby_indices to create entries in the 
        # City_State_Postal_dict -- simultaneously adding the postal code snippet,
        # (ex."NY") from the final two characters in the City_States string i, to 
        # the value, at the specified key, in City_State_Postal_dict
        for i in City_State_groupby_indices:
            City_State_Postal_dict[i] = i[-2:]

        # Looping through the City_State_Postal_dict (keys: City_State)
        for i in City_State_Postal_dict:
            # If the (value: Postal) is in the Final_Weather_dict (keys: Postal)
            if City_State_Postal_dict[i] in Final_Weather_dict:
                # Set the Final_Weather_Dict (key: Postal) at the specified (value: City_State)
                # as the City_State
                Final_Weather_dict[City_State_Postal_dict[i]][i] = i

        # Return the Final_Weather_dict
        return Final_Weather_dict

    ## CALLING FUNCTION: Postal_City_State_Weather_dict_func
    Final_Weather_Data_dict = Postal_City_State_Weather_dict_func(data, Final_Weather_Data_dict)
      
    ## PART 4:
    # Split data into a summer_df and winter_df for the respective timeframes
    # summer_df = April to September
    summer_df = data[(data['month'] >= 4) & (data['month'] <= 9)]
    summer_df = summer_df.copy()
    # winter_df = October to March
    winter_df = data[(data['month'] < 4) | (data['month'] > 9)]
    winter_df = winter_df.copy()


    ## PART 5:
    # Creating a city_state_avg_dict that holds all the mean values, for the
    # respective columns, in the respective season
    summer_city_state_avg = summer_df.groupby('City_State').mean()
    summer_city_state_avg_dict = summer_city_state_avg.to_dict(orient='index')

    winter_city_state_avg = winter_df.groupby('City_State').mean()
    winter_city_state_avg_dict = winter_city_state_avg.to_dict(orient='index')


    ## PART 6:
    def Postal_City_State_Weather_dict_FINAL_func(df, Final_Weather_dict):
        # Instantiate an empty dictionary to hold the City_State as the key and the
        # Weather data as the value
        City_State_Weather_dict = {}

        # Looping through the City_State
        for j in df['City_State']:
            # If it's NOT IN the City_State_Weather_dict
            if j not in City_State_Weather_dict:
                # Instantiate an empty dictionary as the value in City_State_Weather_dict 
                City_State_Weather_dict[j] = {}
        
        # Looping through the City_State in the City_State_Weather_dict 
        for x in City_State_Weather_dict:
            # If it IS IN the summer_city_state_avg_dict 
            if x in summer_city_state_avg_dict:
                # Create an entry called 'summer' in the City_State_Weather_dict value
                # space which holds the weather data from the summer_city_state_avg_dict
                # at the specified City_State
                City_State_Weather_dict[x]['summer'] = summer_city_state_avg_dict[x]

        # Doing the same as above for the winter_city_state_avg_dict 
        for x in City_State_Weather_dict:
            if x in winter_city_state_avg_dict:
                City_State_Weather_dict[x]['winter'] = winter_city_state_avg_dict[x]

        # Looping through the postal codes in Final_Weather_dict  
        for postal in Final_Weather_dict:
            # Looping through the city_states in the postal dictionary
            for city_state in Final_Weather_dict[postal]:
                # If the city_state is in the City_State_Weather_dict 
                if city_state in City_State_Weather_dict:
                    # Set the Final_Weather_dict value for the city_state, in the 
                    # respective postal code, as the weather data with the respective seasons 
                    Final_Weather_dict[postal][city_state] = City_State_Weather_dict[city_state]

        # Return the Final_Weather_Dict
        return Final_Weather_dict
    
    ## CALLING FUNCTION: Postal_City_State_Weather_dict_FINAL_func
    FINAL_Weather_dict = Postal_City_State_Weather_dict_FINAL_func(data, Final_Weather_Data_dict)
    
    ## JSON  
    weather_json = json.dumps(FINAL_Weather_dict)
    
    # return
    return weather_json


## CHECK:
weather_JSON = weather_json_func(weather_data)
weather_JSON